In [15]:
import torch
from torchvision import datasets, transforms
import heapq
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import time
from torch._utils import _accumulate
import numpy as np
from sklearn.cluster import KMeans
import copy
from PIL import Image

from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score

nmi = normalized_mutual_info_score
import numpy as np
batch_size = 250
dataset_size = 70000
train_set = dset.MNIST(root='/home/mrsadeghi/Spectral_clustering_network', train=True,
                       transform=transforms.ToTensor(), download=True)
test_set = dset.MNIST(root='/home/mrsadeghi/Spectral_clustering_network', train=False,
                      transform=transforms.ToTensor(), download=True)

kwargs = {'num_workers': 1}

train1 = torch.utils.data.ConcatDataset([train_set, test_set])

train_loader = torch.utils.data.DataLoader(
    dataset=train1,
    batch_size=batch_size,
    shuffle=False, **kwargs)
X_train = np.zeros([70000,1,28,28])
y_true = np.zeros([70000])
ii = 0
for x, target in train_loader:
    x = x.numpy()
    X_train[ii * batch_size:(ii + 1) * batch_size] = x
    y_true[ii * batch_size:(ii + 1) * batch_size] = target.numpy()
    ii = ii + 1
    print(ii)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [17]:
from __future__ import print_function
import os,sys
os.environ['KERAS_BACKEND'] = 'theano'
os.environ['THEANO_FLAGS']='device=gpu6,lib.cnmem=1,mode=FAST_RUN,floatX=float32,optimizer=fast_compile'
# os.environ['THEANO_FLAGS']='device=cpu,data_format=NHWC'
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
#==============================================================================
# import os
# os.environ['KERAS_BACKEND'] = 'tensorflow'
# os.environ['CUDA_VISIBLE_DEVICES'] = '6'
# os.environ['IMAGE_DIM_ORDERING'] = 'tf'
# import tensorflow as tf
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.3
# tf.Session(config=config)
#==============================================================================
###STL good
import numpy as np
import h5py
from keras.datasets import mnist,cifar10
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input
from keras.layers import Dropout, merge, Lambda, Reshape, Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, AveragePooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.engine.topology import Layer
import matplotlib.pyplot as plt
import scipy.io as sio
import json
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers.advanced_activations import LeakyReLU
from keras.models import load_model
from sys import path
path.append('/home/changjianlong/clustering')
from myMetrics import *

global upper, lower
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.18,
    height_shift_range=0.18,
    channel_shift_range=0.1,
    horizontal_flip=True,
    rescale=0.95,
    zoom_range=[0.85,1.15]
)

class Adaptive(Layer):
    def __init__(self, norm=2.0, learnable = False, **kwargs):
        self.norm = norm
        self.learnable = learnable
        super(Adaptive, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.nb_sample = input_shape[0]
        self.nb_dim = input_shape[1]
        self.learn_norm = K.variable(self.norm)
        if self.learnable == True:
            self.trainable_weights = [self.learn_norm]
        else:
            self.non_trainable_weights1 = [self.learn_norm]
        
    def call(self, x, mask = None):
        y = self.transfer(x)
        return y
        
    def get_output_shape_for(self, input_shape):
        return (input_shape[0],input_shape[1])
        
    def transfer(self, x):
        y = K.pow(K.sum(x**self.learn_norm, axis = 1), 1./self.learn_norm)
        y = K.expand_dims(y, axis = 1)
        y = K.repeat_elements(y, self.nb_dim, axis = 1)
        return x/y

class DotDist(Layer):
    def __init__(self, **kwargs):
        super(DotDist, self).__init__(**kwargs)
        
    def call(self, x, mask = None):
        d = K.dot(x, K.transpose(x))
        return d
        
    def get_output_shape_for(self, input_shape):
        return (input_shape[0],input_shape[0])

class CosineDist(Layer):
    def __init__(self, **kwargs):
        super(CosineDist, self).__init__(**kwargs)
        
    def call(self, x, mask = None):
        x = K.l2_normalize(x, axis =1)
        d = K.dot(x, K.transpose(x))
        return d
        
    def get_output_shape_for(self, input_shape):
        return (input_shape[0],input_shape[0])

#data
nb_classes = 10
img_channels, img_rows, img_cols = 1, 28, 28
# file = h5py.File('/home/changjianlong/datasets/ImageNet96.h5')
# X_train = file['X_train'][:]
# y_true = file['y_train'][:]
# y_true = y_true.astype('int64')
# file.close()

# mean_image0 = np.mean(X_train[:,0,:,:])
# mean_image1 = np.mean(X_train[:,1,:,:])
# mean_image2 = np.mean(X_train[:,2,:,:])
# X_train[:,0,:,:] -= mean_image0
# X_train[:,1,:,:] -= mean_image1
# X_train[:,2,:,:] -= mean_image2

index = np.arange(X_train.shape[0])
np.random.shuffle(index)
X_train = X_train[index].reshape(70000, 28, 28, 1)
y_true = y_true[index]
tempmap = np.copy(index)
#parameters
batch_size = 32
epoch = 50
nb_epoch = 10
upper = 0.99
lower = 0.8
eta = (upper-lower)/epoch/2
nb = 1000
run = 'Clustering'
#run = 'Classification'
nb_samples = [500]
print(run)
#model
#==============================================================================
# X_train = np.transpose(X_train, (0,2,3,1))
#==============================================================================
inp_ = Input(shape=(img_channels, img_rows, img_cols))
x = Convolution2D(64,(3,3),kernel_initializer = 'he_normal',data_format = 'channels_first')(inp_)
x = BatchNormalization(axis = 1)(x)
x = Activation('relu')(x)
x = Convolution2D(64,(3,3),kernel_initializer = 'he_normal',data_format = 'channels_first')(x)
x = BatchNormalization(axis = 1)(x)
x = Activation('relu')(x)
x = Convolution2D(64,(3,3),kernel_initializer = 'he_normal',data_format = 'channels_first')(x)
x = BatchNormalization(axis = 1)(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2,2),data_format = 'channels_first')(x)
x = Convolution2D(128,(3,3),kernel_initializer = 'he_normal',data_format = 'channels_first')(x)
x = BatchNormalization(axis = 1)(x)
x = Activation('relu')(x)
x = Convolution2D(128,(3,3),kernel_initializer = 'he_normal',data_format = 'channels_first')(x)
x = BatchNormalization(axis = 1)(x)
x = Activation('relu')(x)
x = Convolution2D(128,(3,3),kernel_initializer = 'he_normal',data_format = 'channels_first')(x)
x = BatchNormalization(axis = 1)(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2,2),data_format = 'channels_first')(x)
x = Convolution2D(10,1,1,kernel_initializer = 'he_normal',data_format = 'channels_first')(x)
x = BatchNormalization(axis = 1)(x)
x = Activation('relu')(x)
x = AveragePooling2D(pool_size = (2,2),data_format = 'channels_first')(x)
x0 = Flatten()(x)
# x = Convolution2D(256,3,3,init = 'he_normal')(x)
# x = BatchNormalization(mode=2,axis = 1)(x)
# x = Activation('relu')(x)
# x = Convolution2D(256,3,3,init = 'he_normal')(x)
# x = BatchNormalization(mode=2,axis = 1)(x)
# x = Activation('relu')(x)
# x = Convolution2D(256,3,3,init = 'he_normal')(x)
# x = BatchNormalization(mode=2,axis = 1)(x)
# x = Activation('relu')(x)
# x = MaxPooling2D(pool_size = (2,2))(x)
# x = Convolution2D(nb_classes, 1, 1,init = 'he_normal')(x)
# x = BatchNormalization(mode = 2,axis = 1)(x)
# x = Activation('relu')(x)
# x = AveragePooling2D(pool_size = (6,6))(x)
# x0 = Flatten()(x)
x1 = Dense(nb_classes,kernel_initializer = 'he_normal')(x0)
x1 = BatchNormalization()(x1)
x1 = Activation('relu')(x1)
x1 = Dense(nb_classes,kernel_initializer = 'he_normal')(x1)
x1 = BatchNormalization()(x1)
y = Activation('softmax')(x1)
z = Adaptive(norm = 2, name = 'aux_output')(y)
dist = DotDist(name = 'main_output')(z)

norm_l2 = Model(inputs=[inp_], outputs=[x0])
cluster_l1 = Model(inputs=[inp_], outputs=[y])
cluster_l2 = Model(inputs=[inp_], outputs=[z])
model = Model(inputs=[inp_], outputs=[z, dist])


location = str(sys.argv[0])
location = location.replace('.py','.h5')
weight_path = location.replace('.h5','')
print(location)
print(weight_path)
if not os.path.exists(weight_path):
    os.makedirs(weight_path)

if run == 'Clustering':
    acc = []
    ari = []
    nmi = []
    ind = []
    output = []
#     print(X_train.shape)
    Output = cluster_l1.predict(X_train)
    y_pred = np.argmax(Output,axis = 1)
    nmit = NMI(y_true,y_pred)
    arit = ARI(y_true,y_pred)
    acct, indt = ACC(y_true,y_pred)
    model_weight_location = location.replace('.h5','/model_weight_epoch_{}.h5'.format(0))
    cluster_l1.save_weights(model_weight_location)
    print(nmit, arit, acct)
    acc.append(acct)
    ari.append(arit)
    nmi.append(nmit)
    ind.append(indt)
    output.append(Output)
    
    index = np.arange(X_train.shape[0])
    index_loc = np.arange(nb)
    for e in range(epoch):
        np.random.shuffle(index)
        print(lower, upper)
        def my_binary_crossentropy(y_true, y_pred):
            w1 = K.switch(y_true<lower,1,0)
            w2 = K.switch(y_true>upper,1,0)
            w = w1 + w2
            nb_selected = K.sum(w)
            y_true = K.switch(y_true>(upper+lower)/2,1,0)
            return K.sum(w*K.binary_crossentropy(y_true, y_pred), axis=-1)/nb_selected

        model.compile(optimizer=RMSprop(0.001), 
              loss={'main_output':my_binary_crossentropy,'aux_output':'binary_crossentropy'}, 
              loss_weights={'main_output':1,'aux_output':1})
        
        if X_train.shape[0]>nb:
            for i in range(X_train.shape[0]//nb):
                Xbatch = X_train[index[np.arange(i*nb,(i+1)*nb)]]
                Y = cluster_l2.predict(Xbatch)
                Ybatch = np.dot(Y,Y.T)
                for k in range(nb_epoch):
                    np.random.shuffle(index_loc)
                    for j in range(Xbatch.shape[0]//batch_size):
                        address = index_loc[np.arange(j*batch_size,(j+1)*batch_size)]
                        X_batch = Xbatch[address]
                        Y_batch = Ybatch[address,:][:,address]
                        Y_ = Y[address]
                        sign = 0
                        for X_batch_i in datagen.flow(X_batch, batch_size=batch_size,shuffle=False):
                            loss = model.train_on_batch([X_batch_i],[Y_, Y_batch])
                            sign += 1
                            if sign>1:
                                break
                if i%10==0:
                    print('Epoch: %d, batch: %d/%d, loss: %f, loss1: %f, loss2: %f, nb1: %f'%
                          (e+1,i+1,X_train.shape[0]//nb,loss[0],loss[1],loss[2],np.mean(Ybatch)))
        else:
            print('error')
        upper = upper - eta
        lower = lower + eta
        Output = cluster_l1.predict(X_train)
        y_pred = np.argmax(Output,axis = 1)
        nmit = NMI(y_true,y_pred)
        arit = ARI(y_true,y_pred)
        acct, indt = ACC(y_true,y_pred)
        model_weight_location = location.replace('.h5','/model_weight_epoch_{}.h5'.format(e+1))
        cluster_l1.save_weights(model_weight_location)
        print(nmit, arit, acct)
        acc.append(acct)
        ari.append(arit)
        nmi.append(nmit)
        ind.append(indt)
        output.append(Output)
        
        if os.path.exists(location):
            os.remove(location)
        file = h5py.File(location,'w')
        file.create_dataset('acc',data = acc)
        file.create_dataset('nmi',data = nmi)
        file.create_dataset('ari',data = ari)
        file.create_dataset('ind',data = ind)
        file.create_dataset('output',data = output)
        file.create_dataset('tempmap',data = tempmap)
        file.close()
else:
    for nb_sample in nb_samples:
        X = X_train[0:nb_sample]
        Y = np_utils.to_categorical(y_true[0:nb_sample], nb_classes)
        X_test = X_train[-1000:-1]
        Y_test = np_utils.to_categorical(y_true[-1000:-1], nb_classes)
        cluster_l1.fit(X, Y, nb_epoch=100,validation_data=(X_test, Y_test), verbose=2)
#==============================================================================
#         cluster_l1.fit_generator(datagen.flow(X, Y, batch_size=32), 
#                               nb_sample // 32,nb_epoch=100, verbose=2, validation_data=(X_test, Y_test))
#==============================================================================
    model_weight_location = location.replace('.h5','/model_weight_epoch_{}.h5'.format(e))
    cluster_l1.load_weights(model_weight_location)
    f = h5py.File(location)
    acc = f['acc'][:]
    nmi = f['nmi'][:]
    ari = f['ari'][:]
    ind = f['ind'][:]
    output = f['output'][:]
    tempmap = f['tempmap'][:]
    f.close()
    Output = cluster_l1.predict(X_train)
    y_pred = np.argmax(Output,axis = 1)
    acc, ind = ACC(y_true,y_pred)
    
    y_trasfer = np.ones_like(y_true)
    for i in range(nb_classes):
        index = np.where(y_true == i)
        indext = np.where(ind[:, 1]==i)
        y_trasfer[index] = ind[indext, 0]
    acct, indt = ACC(y_trasfer,y_pred)
    assert(acc==acct)
    
    for nb_sample in nb_samples:
        X = X_train[0:nb_sample]
        Y = np_utils.to_categorical(y_trasfer[0:nb_sample], nb_classes)
        X_test = X_train[-1000:-1]
        Y_test = np_utils.to_categorical(y_trasfer[-1000:-1], nb_classes)
        cluster_l1.fit(X, Y, nb_epoch=100,validation_data=(X_test, Y_test), verbose=2)
#==============================================================================
#         cluster_l1.fit_generator(datagen.flow(X, Y, batch_size=32), 
#                               nb_sample // 32,nb_epoch=100, verbose=2,validation_data=(X_test, Y_test))
#==============================================================================


Clustering
c:\users\wykzh\.virtualenvs\dsec-mra4zkvh\lib\site-packages\ipykernel_launcher.h5
c:\users\wykzh\.virtualenvs\dsec-mra4zkvh\lib\site-packages\ipykernel_launcher


ValueError: in user code:

    c:\users\wykzh\.virtualenvs\dsec-mra4zkvh\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    c:\users\wykzh\.virtualenvs\dsec-mra4zkvh\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\wykzh\.virtualenvs\dsec-mra4zkvh\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\wykzh\.virtualenvs\dsec-mra4zkvh\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\wykzh\.virtualenvs\dsec-mra4zkvh\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\users\wykzh\